<a href="https://colab.research.google.com/github/Jacques-Wolbeck/Processamento-Digital-Imagens-UFAL/blob/main/atividades/image_stitching/image_stitching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
from imutils import paths
import numpy as np
import imutils
import cv2


drive.mount('/content/drive')


images = []

images.append(cv2.imread('/content/drive/MyDrive/images/ic1.jpg'))
images.append(cv2.imread('/content/drive/MyDrive/images/ic2.jpg'))
images.append(cv2.imread('/content/drive/MyDrive/images/ic3.jpg'))
images.append(cv2.imread('/content/drive/MyDrive/images/ic4.jpg'))

print("Stitching images...")
stitcher = cv2.createStitcher() if imutils.is_cv3() else cv2.Stitcher_create()
(status, stitched) = stitcher.stitch(images)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Stitching images...


In [ ]:
if status == 0:
  stitched = cv2.copyMakeBorder(stitched, 10, 10, 10, 10, cv2.BORDER_CONSTANT, (0,0,0))
  gray = cv2.cvtColor(stitched, cv2.COLOR_BGR2GRAY)
  thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY) [1]

  cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  cnts = imutils.grab_contours(cnts)
  c = max(cnts, key= cv2.contourArea)

  mask = np.zeros(thresh.shape, dtype="uint8")
  (x, y, w, h) = cv2.boundingRect(c)
  cv2.rectangle(mask, (x,y), (x + w, y + h), 255, -1)

  minRect = mask.copy()
  sub = mask.copy()

  while cv2.countNonZero(sub) > 0:
    minRect = cv2.erode(minRect, None)
    sub = cv2.subtract(minRect, thresh)

  cnts = cv2.findContours(minRect.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  cnts = imutils.grab_contours(cnts)
  c = max(cnts, key=cv2.contourArea)
  (x, y, w, h) = cv2.boundingRect(c)

  stitched = stitched[y:y + h, x:x + w]

  cv2.imwrite('/content/drive/MyDrive/images/ic.jpg', stitched)

else:
	print("Image stitching failed ({})".format(status))